In [ ]:
from hhl_openai_agents_sdk.models import get_gemini
from dotenv import load_dotenv
import nest_asyncio

nest_asyncio.apply()
load_dotenv()
model = get_gemini()

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


In [6]:
from agents import Agent, InputGuardrail,GuardrailFunctionOutput, Runner
from pydantic import BaseModel

class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
    model=model,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=model,
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=model,
)


async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    if not final_output.is_homework:
        print("Guardrail triggered, reason:", final_output.reasoning)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    model=model,
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

async def main():
    result = await Runner.run(triage_agent, "who was the first president of the united states?")
    print(result.final_output)

    result = await Runner.run(triage_agent, "what is life")
    print(result.final_output)


In [7]:
await main()

The first president of the United States was **George Washington**. He served from 1789 to 1797.

Here's some important context:

*   **American Revolutionary War:** Washington was the commander of the Continental Army during the American Revolutionary War (1775-1783). His leadership was crucial in securing independence from British rule.
*   **Constitutional Convention:** After the war, Washington presided over the Constitutional Convention of 1787. This convention drafted the United States Constitution, which established the framework for the new government.
*   **Unanimous Election:** Washington was unanimously elected as president by the Electoral College in 1789 and again in 1792. This highlights the immense respect and trust he commanded.
*   **Setting Precedents:** As the first president, Washington established many important precedents for the office. He focused on creating a strong, stable, and unified nation. Some key examples include:

    *   Establishing a cabinet of advis

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire